In [ ]:
%pip install rouge_score
from transformers import pipeline
from datasets import load_dataset
from rouge_score import rouge_scorer
import pandas as pd

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e3d9890613c1bc69b294a812685ddf2232734677f18221edfd6255c52193cd7c
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
# Load CNN/DailyMail test dataset
cnn_dailymail = load_dataset("cnn_dailymail", "3.0.0")

# Initialize models
models = {
    "PEGASUS": pipeline('summarization', model="google/pegasus-cnn_dailymail"),
    "BART": pipeline('summarization', model="facebook/bart-large-cnn")
}

# Inference parameters
gen_kwargs = {
    'length_penalty': 0.8,  # Suitable for CNN/DailyMail's multi-sentence summaries
    'num_beams': 8,
    'max_length': 256,
    'min_length': 50
}

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
# Summarize and evaluate 2 CNN/DailyMail articles
results = []
for i in range(2):
    article = cnn_dailymail["test"][i]["article"]
    reference = cnn_dailymail["test"][i]["highlights"]

    for model_name, pipe in models.items():
        try:
            summary = pipe(article, **gen_kwargs)[0]['summary_text']
            scores = scorer.score(reference, summary)
            results.append({
                'Article': f"CNN/DailyMail {i+1}",
                'Model': model_name,
                'ROUGE-1': scores['rouge1'].fmeasure,
                'ROUGE-2': scores['rouge2'].fmeasure,
                'ROUGE-L': scores['rougeL'].fmeasure,
                'Reference Summary': reference,
                'Generated Summary': summary
            })
        except Exception as e:
            print(f"Error processing Article {i+1} with {model_name}: {e}")

In [ ]:
# Custom articles
custom_articles = [
    {
        "name": "Custom Article 1 (UK Travel)",
        "text": """
The UK government has announced new travel restrictions due to rising COVID-19 cases. Starting next week, all travelers entering the UK must provide a negative test result taken within 72 hours of departure. Additionally, a mandatory 10-day quarantine will be enforced for arrivals from high-risk countries, with options for a test-to-release scheme after five days. The measures aim to curb the spread of new variants, but critics argue they may disrupt international travel and trade.
"""
    },
    ]

In [ ]:
# Summarize custom articles
custom_results = []
for article_info in custom_articles:
    for model_name, pipe in models.items():
        try:
            summary = pipe(article_info["text"], **gen_kwargs)[0]['summary_text']
            custom_results.append({
                'Article': article_info["name"],
                'Model': model_name,
                'Generated Summary': summary
            })
        except Exception as e:
            print(f"Error processing {article_info['name']} with {model_name}: {e}")


Your max_length is set to 256, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 256, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)


In [ ]:
# Display results
results_df = pd.DataFrame(results)
custom_results_df = pd.DataFrame(custom_results)

print("\n--- ROUGE Scores for CNN/DailyMail Articles ---")
print(results_df[['Article', 'Model', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L']])

print("\n--- Summaries for CNN/DailyMail Article 1 ---")
print("Article:\n", cnn_dailymail["test"][0]["article"])
print("\nReference Summary:\n", results_df[results_df['Article'] == "CNN/DailyMail 1"]['Reference Summary'].iloc[0])
for model_name in models:
    print(f"\n{model_name} Summary:\n", results_df[(results_df['Article'] == "CNN/DailyMail 1") & (results_df['Model'] == model_name)]['Generated Summary'].iloc[0])

print("\n--- Custom Article Summaries ---")
for article_name in [ca["name"] for ca in custom_articles]:
    print(f"\n{article_name}:")
    for model_name in models:
        summary = custom_results_df[(custom_results_df['Article'] == article_name) & (custom_results_df['Model'] == model_name)]['Generated Summary']
        if not summary.empty:
            print(f"{model_name} Summary:\n", summary.iloc[0])


--- ROUGE Scores for CNN/DailyMail Articles ---
           Article    Model   ROUGE-1   ROUGE-2   ROUGE-L
0  CNN/DailyMail 1  PEGASUS  0.470588  0.337349  0.400000
1  CNN/DailyMail 1     BART  0.538462  0.315789  0.435897
2  CNN/DailyMail 2  PEGASUS  0.510638  0.282609  0.446809
3  CNN/DailyMail 2     BART  0.488372  0.285714  0.488372

--- Summaries for CNN/DailyMail Article 1 ---
Article:
 (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into 